In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA

# Step 1: Data Preprocessing
data = pd.read_csv('train.csv')
data1 = pd.read_csv('test.csv')
data.drop(columns=['ID','Candidate','Constituency ∇'], inplace=True)
data1.drop(columns=['ID','Candidate','Constituency ∇'], inplace=True)

def convert_to_lakhs(value):
    value = str(value)
    if value == "0":
        return 0
    elif 'Crore+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number * 100)
    elif 'Lac+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number)
    elif 'Thou+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number / 100)
    elif 'Hund+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number / 1000)
    else:
        return int(value)

data["Total Assets"] = data["Total Assets"].apply(convert_to_lakhs)
data["Liabilities"] = data["Liabilities"].apply(convert_to_lakhs)
data1["Total Assets"] = data1["Total Assets"].apply(convert_to_lakhs)
data1["Liabilities"] = data1["Liabilities"].apply(convert_to_lakhs)

# Perform label encoding for Education column

from sklearn.preprocessing import LabelEncoder

# Encode the target variable
label_encoder = LabelEncoder()
data['Education'] = label_encoder.fit_transform(data['Education'])

# data['Education'] = data['Education'].astype('category').cat.codes
X = data.drop(columns=['Education'])
y = data['Education']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# y_check = label_encoder.inverse_transform(y_test)

label_relative_frequencies = []
total_train_rows = len(X)

# Iterate through each label
for label in range(10):
    label_dataset_rows = (y == label).sum()
    label_relative_frequency = label_dataset_rows / total_train_rows
    label_relative_frequencies.append(label_relative_frequency)

# print(label_relative_frequencies)

y = pd.get_dummies(y, columns=['Education'])
X = pd.get_dummies(X, columns=['Party','state'])
# X_train.drop(columns=['Party_CPI','Party_TDP','Party_Tipra Motha Party'],inplace=True)
data1 = pd.get_dummies(data1, columns=['Party','state'])
# data1 = pd.get_dummies(data1, columns=['Party','state'])

# Step 3: Train a Random Forest classifier
rf_classifier = RandomForestClassifier( n_estimators=91, max_depth=25, max_features='log2', min_samples_split=7, min_samples_leaf=1, random_state=42)
rf_classifier.fit(X, y)

predictions = []
for i in range(len(data1)):
    y_pred_proba = rf_classifier.predict_proba(data1.iloc[[i]])
    result_list = [(arr[0][1] * label_relative_frequencies[i]) for i, arr in enumerate(y_pred_proba)]
    max_index = result_list.index(max(result_list))
    predictions.append(max_index)


predictions = label_encoder.inverse_transform(predictions)
# print(predictions)

# f1 = f1_score(y_check, predictions, average='weighted')
# print("F1 Score:", f1)
df_pred = pd.DataFrame(predictions, columns=["Education"])
df_pred.to_csv('submission.csv', index_label='ID')


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA

# Step 1: Data Preprocessing
data = pd.read_csv('train.csv')
data1 = pd.read_csv('test.csv')
data.drop(columns=['ID','Candidate','Constituency ∇'], inplace=True)
data1.drop(columns=['ID','Candidate','Constituency ∇'], inplace=True)

def convert_to_lakhs(value):
    value = str(value)
    if value == "0":
        return 0
    elif 'Crore+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number * 100)
    elif 'Lac+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number)
    elif 'Thou+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number / 100)
    elif 'Hund+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number / 1000)
    else:
        return int(value)

data["Total Assets"] = data["Total Assets"].apply(convert_to_lakhs)
data["Liabilities"] = data["Liabilities"].apply(convert_to_lakhs)
data1["Total Assets"] = data1["Total Assets"].apply(convert_to_lakhs)
data1["Liabilities"] = data1["Liabilities"].apply(convert_to_lakhs)

# Perform label encoding for Education column

from sklearn.preprocessing import LabelEncoder

# Encode the target variable
label_encoder = LabelEncoder()
data['Education'] = label_encoder.fit_transform(data['Education'])

label_relative_frequencies = []
total_train_rows = len(data)

# Iterate through each label
for label in range(10):
    label_dataset_rows = (data['Education'] == label).sum()
    label_relative_frequency = label_dataset_rows / total_train_rows
    label_relative_frequencies.append(label_relative_frequency)

print(label_relative_frequencies)

# data['Education'] = data['Education'].astype('category').cat.codes

data = pd.get_dummies(data, columns=['Party','state','Education'])
# data1 = pd.get_dummies(data1, columns=['Party','state'])


X = data.drop(columns=['Education_0', 'Education_1', 'Education_2', 'Education_3', 'Education_4', 'Education_5', 'Education_6', 'Education_7', 'Education_8', 'Education_9'])
y = data[['Education_0', 'Education_1', 'Education_2', 'Education_3', 'Education_4', 'Education_5', 'Education_6', 'Education_7', 'Education_8', 'Education_9']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
predictions = []
for i in range(len(X_test)):
    y_pred_proba = rf_classifier.predict_proba(X_test.iloc[[i]])
    result_list = [(arr[0][1] * label_relative_frequencies[i]) for i, arr in enumerate(y_pred_proba)]
    max_index = result_list.index(max(result_list))
    predictions.append(max_index)


predictions = label_encoder.inverse_transform(predictions)
print(predictions)

# Print the original labels
# Step 4: Multiply predicted probabilities with occurrence probabilities of each label in the training dataset
# Calculate occurrence probabilities
# label_occurrence_probabilities = y_train.mean()

# # Multiply predicted probabilities with occurrence probabilities
# probability_products = y_pred_proba * label_occurrence_probabilities.values

# # Step 5: Create a DataFrame to store the probabilities
# labels = ['Education_0', 'Education_1', 'Education_2', 'Education_3', 'Education_4',
#           'Education_5', 'Education_6', 'Education_7', 'Education_8', 'Education_9']

# prob_df = pd.DataFrame(data=probability_products, columns=labels)

# # Step 6: Print the DataFrame showing the probability of each row of the test data for each label
# print(prob_df)

# X = data.drop(columns=['Education'])
# y = data['Education']

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # n_components = 30  # Specify the number of principal components
# # pca = PCA(n_components=n_components)
# # X_train = pca.fit_transform(X_train)
# # X_test = pca.transform(X_test)

# # Step 2: Training the Random Forest Classifier
# rf_classifier = RandomForestClassifier(n_estimators=86,
#                                         criterion='gini',
#                                         max_depth=26,
#                                         min_samples_split=2,
#                                         max_features='log2',
#                                         random_state=42)
# rf_classifier.fit(X_train, y_train)

# # Step 3: Model Evaluation
# y_pred = rf_classifier.predict(X_test)
# f1 = f1_score(y_test, y_pred, average='weighted')
# print("F1 Score:", f1)
# y_pred = label_encoder.inverse_transform(y_pred)
# Step 4: Data Visualization - Plotting code goes here

# Step 5: Generate LaTeX Report - Details about the model, hyperparameters, evaluation metrics, and data visualization results.


[0.11024769305488101, 0.1694997571636717, 0.004371053909664886, 0.03788246721709568, 0.02525497814473045, 0.2578921806702283, 0.16464303059737737, 0.006799417192812044, 0.013598834385624089, 0.20981058766391453]
['Post Graduate' '12th Pass' '12th Pass' 'Post Graduate' '12th Pass'
 'Post Graduate' 'Graduate' 'Graduate' 'Graduate' 'Graduate' 'Graduate'
 'Graduate' 'Graduate Professional' 'Graduate' '12th Pass' 'Graduate'
 'Graduate Professional' '12th Pass' 'Graduate' '12th Pass' 'Graduate'
 'Graduate' 'Post Graduate' 'Post Graduate' 'Graduate' 'Graduate'
 'Graduate' 'Graduate' 'Graduate' 'Post Graduate' 'Graduate Professional'
 '10th Pass' 'Graduate' 'Graduate' '12th Pass' '12th Pass' 'Graduate'
 'Post Graduate' 'Graduate' 'Graduate' 'Graduate' 'Graduate Professional'
 '12th Pass' 'Graduate Professional' 'Post Graduate' '10th Pass'
 'Graduate' '12th Pass' 'Post Graduate' '12th Pass' 'Graduate' 'Graduate'
 'Post Graduate' 'Post Graduate' 'Graduate' 'Graduate' '12th Pass'
 'Graduate' '10t

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Step 1: Data Preprocessing
data = pd.read_csv('train.csv')
data1 = pd.read_csv('test.csv')
data.drop(columns=['Candidate','Constituency ∇'], inplace=True)
data1.drop(columns=['Candidate','Constituency ∇'], inplace=True)

def convert_to_lakhs(value):
    value = str(value)
    if value == "0":
        return 0
    elif 'Crore+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number * 100)
    elif 'Lac+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number)
    elif 'Thou+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number / 100)
    elif 'Hund+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number / 1000)
    else:
        return int(value)

data["Total Assets"] = data["Total Assets"].apply(convert_to_lakhs)
data["Liabilities"] = data["Liabilities"].apply(convert_to_lakhs)
data1["Total Assets"] = data1["Total Assets"].apply(convert_to_lakhs)
data1["Liabilities"] = data1["Liabilities"].apply(convert_to_lakhs)

# Perform label encoding for Education column

from sklearn.preprocessing import LabelEncoder

# Encode the target variable
label_encoder = LabelEncoder()
data['Education'] = label_encoder.fit_transform(data['Education'])

# data['Education'] = data['Education'].astype('category').cat.codes

data = pd.get_dummies(data, columns=['Party','state'])
data1 = pd.get_dummies(data1, columns=['Party','state'])

X = data.drop(columns=['Education'])
y = data['Education']

# Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)
data1 = scaler.transform(data1)

# n_components = 30  # Specify the number of principal components
# pca = PCA(n_components=n_components)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

# Step 2: Training the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=85,
                                        criterion='gini',
                                        max_depth=30,
                                        min_samples_split=2,
                                        min_samples_leaf=1,
                                        min_weight_fraction_leaf=0.0,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        # min_impurity_split=None,
                                        bootstrap=True,
                                        oob_score=False,
                                        n_jobs=None,
                                        random_state=42,
                                        verbose=1,
                                        warm_start=False,
                                        class_weight=None,
                                        ccp_alpha=0.0,
                                        max_samples=None)
rf_classifier.fit(X, y)

# Step 3: Model Evaluation
y_pred = rf_classifier.predict(data1)
# f1 = f1_score(y_test, y_pred, average='weighted')
# print("F1 Score:", f1)
y_pred = label_encoder.inverse_transform(y_pred)
# Step 4: Data Visualization - Plotting code goes here

# Step 5: Generate LaTeX Report - Details about the model, hyperparameters, evaluation metrics, and data visualization results.


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [ ]:
df_pred = pd.DataFrame(y_pred, columns=["Education"])
df_pred.to_csv('submission.csv', index_label='ID')

In [ ]:
y_pred

array(['12th Pass', 'Graduate Professional', 'Graduate', ...,
       'Post Graduate', 'Graduate Professional', '12th Pass'],
      dtype=object)